## Data Handling Script

**Purpose:**  (1) Extract necessary metrics from all multi-domain studies and export to `.csv` for further exploration, (2) calculate statistics on amount of data analyzed

Author: Saurish Srivastava @ [Subjectivity Lab](https://subjectivity.sites.northeastern.edu/)

In [ ]:
# install packages
!pip3 install numpy
!pip3 install pandas
!pip3 install seaborn
!pip3 install matplotlib
!pip3 install scipy
!pip3 install arviz
!pip3 install git+https://github.com/embodied-computation-group/metadPy.git

In [1]:
# imports
import numpy as np
import pandas as pd
import arviz as az
import scipy.stats as st
from metadPy.mle import fit_metad, metad
from metadPy import load_dataset
from metadPy.utils import trials2counts
from metadPy.plotting import plot_confidence, plot_roc
from metadPy.sdt import scores, rates, dprime, criterion, roc_auc
import matplotlib.pyplot as plt

from functools import partial, reduce

## Define data handling functions

In [8]:
def getGroupData(data, nRatings, export):
    """
    For dataframe {data} & int {nRatings} & string {export}: finds scores, rates, dprime, metad, 
    criterion, mratio, and mdiff for each subject and export it to {export} filepath.
    Returns: a dataframe with above variables formatted as {domain}_{variable} and exports to {export} location
    """
    # get full domain list
    domains = list(data['Task'].unique())
    
    temp_dict = {}
    
    # iterate through each domain
    for domain in domains:
        domain_data = data.loc[data['Task'] == domain] # only get proper domain
        domain_data = domain_data.reset_index(drop=True) # reset indexes
        # iterate through each subject in the data
        temp_list = []
        print(f"{domain.capitalize()}\n------------")
        for i in list(domain_data['Subject'].unique()):
            temp_data = domain_data.loc[domain_data['Subject'] == i]
            [nR_S1, nR_S2] = trials2counts(data=temp_data.copy(), stimuli="Stimuli", responses="Responses",
                                   confidence="Confidence", nRatings=nRatings, padding=True)
            # if metad function does not return error: continue with this subject, else: skip subject
            try:
                temp_fit = fit_metad(nR_S1,nR_S2, nRatings=nRatings, nCriteria=int(2 * nRatings - 1))
            except:
                continue
            else:
                temp_scores = scores(data=temp_data)
                print(f"Scores: {temp_scores}")
                temp_rates = rates(data=temp_data)
                print(f"Rates: {temp_rates}")            
                temp_criterion = criterion(data=temp_data)
                print(f"Criterion: {temp_criterion}")                 
                print(f"DPrime: {temp_fit['dprime']}")
                print(f"MetaD: {temp_fit['meta_d']}")                        
                print(f"MRatio: {temp_fit['m_ratio']}")
                print(f"MDiff: {temp_fit['m_diff']}")

                print(f"Done: Subject {i}")
                
                # add all temp variables to list to store them
                temp_list.append([i, temp_scores, temp_rates, temp_fit['dprime'], temp_fit['meta_d'],
                                  temp_criterion, temp_fit['m_ratio'], temp_fit['m_diff']])

        # create pandas dataframe from list data
        groupData = pd.DataFrame(temp_list, columns=['Index', domain + '_scores', domain + '_rates',
                                               domain + '_dprime', domain + '_metad', domain + '_criterion',
                                                     domain + '_mratio', domain + '_mdiff'])
        # change index name
        groupData = groupData.set_index('Index')
        groupData.index.name="Subject"
        
        # add to dictionary
        temp_dict[domain] = groupData
        
        print(f"------------\nCompleted: {domain.capitalize()}\n------------")
    
    # merge data
    my_reduce = partial(pd.merge, on='Subject', how='outer')
    fullData = reduce(my_reduce, temp_dict.values())
    
    # export data
    
    fullData.to_csv('../exports/analysis_' + export + '.csv')
    
    print(f"Exported: 'analysis_{export}.csv'")
    
    return fullData

In [ ]:
def getTotalTrialsAnalyzed(data, nRatings):
    """
    For dataframe 'data': find out the number of trials that are analyzed and the number of
    trials that are excluded during metad calculations
    Prints: # of subjects, # of recorded trials, # of analyzed and non-analyzed trials,
    # of subjects not analyzed, and percentage accounted for & their difference in %
    """
    
    domains = list(data['Task'].unique())
    temp_list = []
    subjectCounter = 0
    skipped_subject = []
    for domain in domains: 
        domain_data = data.loc[data['Task'] == domain] # only get proper domain
        domain_data = domain_data.reset_index(drop=True) # reset indexes
        
        # iterate through each subject in the data
        for i in list(domain_data['Subject'].unique()):
            temp_data = domain_data.loc[domain_data['Subject'] == i]
            [nR_S1, nR_S2] = trials2counts(data=temp_data.copy(), stimuli="Stimuli", responses="Responses",
                       confidence="Confidence", nRatings=nRatings, padding=True)
            # if metad function does not return error, continue with this subject
            try:
                fit_metad(nR_S1,nR_S2, nRatings=nRatings, nCriteria=int(2 * nRatings - 1))
            except:
                skipped_subject.append("Subject " + str(i) + " @ " + domain)
                continue
            else:
                subjectCounter += 1
                temp_list.append(temp_data.shape[0])
        
        print("Completed: " + domain)

    # calculate stats
    percentageAccounted = 100*(sum(temp_list)/data.shape[0])
    difference = 100 * (1 - (sum(temp_list)/data.shape[0]))
    
    print("Number of subjects: " + str(subjectCounter))
    print("Number of recorded trials: " + str(data.shape[0]))
    print("Number of analyzed trials: " + str(sum(temp_list)))
    print("Number of non-analyzed trials: " + str(int(data.shape[0] - sum(temp_list))))
    print("Subjects not analyzed (@ domain): " + str(skipped_subject))
    print("Percentage accounted for: " + str(percentageAccounted) + "%")
    print("Difference (%): " + str(difference) + "%")

## Analysis
### `Mazancieux_2018`

In [50]:
# read in data
data = pd.read_csv("../Confidence Database/data_Mazancieux_2018.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Task'])
data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses"})

# drop all NaNs
data = data.dropna().reset_index(drop=True)

# change dataset's 2s->1s and 1s->0s bc metadpy works with 0s and 1s
data['Stimuli'] = data['Stimuli'].map({1:0, 2:1})
data['Responses'] = data['Responses'].map({1:0, 2:1})

# add 1 to each confidence rating
data['Confidence'] = data['Confidence'] + 1

# create an accuracy column bc it is easier to work with
data['Accuracy'] = np.where((data['Stimuli'] == data['Responses']), 1, 0)

# get data in order by subject
data = data.sort_values(by=['Subject']).reset_index(drop=True)

In [4]:
getGroupData(data=data, nRatings=11, export='Mazancieux_2018')

Vp
------------
Done: Subject 1
Done: Subject 2
Done: Subject 3
Done: Subject 4
Done: Subject 5
Done: Subject 6
Done: Subject 7
Done: Subject 8
Done: Subject 9
Done: Subject 10
Done: Subject 11
Done: Subject 12
Done: Subject 13
Done: Subject 14
Done: Subject 15
Done: Subject 16
Done: Subject 17
Done: Subject 18
Done: Subject 19
Done: Subject 20
Done: Subject 21
Done: Subject 22
Done: Subject 23
Done: Subject 24
Done: Subject 25
Done: Subject 26
Done: Subject 27
Done: Subject 28
Done: Subject 29
Done: Subject 30
Done: Subject 31
Done: Subject 32
Done: Subject 33
Done: Subject 34
Done: Subject 36
Done: Subject 37
Done: Subject 38
Done: Subject 39
Done: Subject 40
Done: Subject 41
Done: Subject 42
Done: Subject 43
Done: Subject 44
Done: Subject 45
Done: Subject 46
Done: Subject 47
Done: Subject 48
Done: Subject 49
Done: Subject 50
Done: Subject 51
Done: Subject 52
Done: Subject 53
Done: Subject 54
Done: Subject 55
Done: Subject 56
Done: Subject 57
Done: Subject 58
Done: Subject 59
Done: S

/opt/anaconda3/lib/python3.8/site-packages/metadPy/mle.py:738: RuntimeWarning: divide by zero encountered in double_scalars
  guess.extend(list(t2c1 - (meta_d1 * (t1c1 / d1))))
/opt/anaconda3/lib/python3.8/site-packages/metadPy/mle.py:738: RuntimeWarning: invalid value encountered in double_scalars
  guess.extend(list(t2c1 - (meta_d1 * (t1c1 / d1))))


Done: Subject 79
Done: Subject 80
Done: Subject 81
Done: Subject 82
Done: Subject 83
Done: Subject 84
Done: Subject 85
Done: Subject 86
Done: Subject 87
Done: Subject 88
Done: Subject 89
Done: Subject 90
Done: Subject 91
Done: Subject 92
Done: Subject 93
Done: Subject 94
Done: Subject 95
Done: Subject 96
Done: Subject 97
Done: Subject 98
Done: Subject 99
Done: Subject 100
Done: Subject 101
Done: Subject 102
Done: Subject 103
Done: Subject 104
Done: Subject 105
Done: Subject 106
Done: Subject 107
Done: Subject 108
Done: Subject 109
Done: Subject 110
Done: Subject 111
Done: Subject 112
Done: Subject 113
Done: Subject 114
Done: Subject 115
Done: Subject 117
Done: Subject 118
Done: Subject 119
Done: Subject 120
Done: Subject 121
Done: Subject 122
Done: Subject 123
Done: Subject 124
Done: Subject 125
Done: Subject 126
Done: Subject 127
Done: Subject 128
Done: Subject 129
Done: Subject 130
Done: Subject 131
Done: Subject 132
Done: Subject 133
Done: Subject 134
Done: Subject 135
Done: Subject

Done: Subject 1
Done: Subject 2
Done: Subject 3
Done: Subject 4
Done: Subject 5
Done: Subject 6
Done: Subject 7
Done: Subject 8
Done: Subject 9
Done: Subject 10
Done: Subject 11
Done: Subject 12
Done: Subject 13
Done: Subject 14
Done: Subject 15
Done: Subject 16
Done: Subject 17
Done: Subject 18
Done: Subject 19
Done: Subject 20
Done: Subject 21
Done: Subject 22
Done: Subject 23
Done: Subject 24
Done: Subject 25
Done: Subject 26
Done: Subject 27
Done: Subject 28
Done: Subject 29
Done: Subject 30
Done: Subject 31
Done: Subject 32
Done: Subject 33
Done: Subject 34
Done: Subject 35
Done: Subject 36
Done: Subject 37
Done: Subject 38
Done: Subject 39
Done: Subject 40
Done: Subject 41
Done: Subject 42
Done: Subject 43
Done: Subject 44
Done: Subject 45
Done: Subject 46
Done: Subject 47
Done: Subject 48
Done: Subject 49
Done: Subject 50
Done: Subject 51
Done: Subject 52
Done: Subject 53
Done: Subject 54
Done: Subject 55
Done: Subject 56
Done: Subject 57
Done: Subject 58
Done: Subject 59
Done: 

VP_scores                                   VP_rates  \
Subject                                                               
1         (17, 5, 5, 13)   (0.7727272727272727, 0.2777777777777778)   
2        (16, 3, 11, 10)   (0.8421052631578947, 0.5238095238095238)   
3         (17, 4, 4, 15)  (0.8095238095238095, 0.21052631578947367)   
4         (16, 4, 5, 15)                                (0.8, 0.25)   
5         (18, 6, 3, 13)                             (0.75, 0.1875)   
...                  ...                                        ...   
35                   NaN                                        NaN   
78                   NaN                                        NaN   
116                  NaN                                        NaN   
139                  NaN                                        NaN   
181                  NaN                                        NaN   

         VP_dprime  VP_metad  VP_criterion  VP_mratio  VP_mdiff  \
Subject                                                           
1         1.264025  0.646595     -0.079201   0.511537 -0.617430   
2         0.877590  0.382232     -0.531433   0.435548 -0.495358   
3         1.580910  0.840950     -0.035773   0.531941 -0.739959   
4         1.429123  1.077677     -0.083566   0.754083 -0.351446   
5         1.464137  1.265364      0.106328   0.864239 -0.198773   
...            ...       ...           ...        ...       ...   
35             NaN       NaN           NaN        NaN       NaN   
78             NaN       NaN           NaN        NaN       NaN   
116            NaN       NaN           NaN        NaN       NaN   
139            NaN       NaN           NaN        NaN       NaN   
181            NaN       NaN           NaN        NaN       NaN   

              SM_scores                                   SM_rates  SM_dprime  \
Subject                                                                         
1        (14, 6, 5, 15)                                (0.7, 0.25)   1.134685   
2        (21, 0, 2, 17)  (0.9761904761904762, 0.10526315789473684)   3.150773   
3        (16, 5, 9, 10)  (0.7619047619047619, 0.47368421052631576)   0.737197   
4        (11, 8, 9, 12)  (0.5789473684210527, 0.42857142857142855)   0.360866   
5        (17, 6, 3, 14)  (0.7391304347826086, 0.17647058823529413)   1.471929   
...                 ...                                        ...        ...   
35       (14, 3, 7, 16)  (0.8235294117647058, 0.30434782608695654)   1.350411   
78       (16, 5, 7, 12)   (0.7619047619047619, 0.3684210526315789)   0.993129   
116      (14, 7, 4, 15)  (0.6666666666666666, 0.21052631578947367)   1.165398   
139      (21, 2, 4, 13)  (0.9130434782608695, 0.23529411764705882)   1.932651   
181       (9, 5, 8, 18)   (0.6428571428571429, 0.3076923076923077)   0.822943   

         ...  EM_criterion  EM_mratio  EM_mdiff       EF_scores  \
Subject  ...                                                      
1        ...      0.256016   1.838230  1.227691  (15, 4, 1, 20)   
2        ...     -0.319957   0.665227 -0.865200  (17, 0, 0, 23)   
3        ...     -0.391543   2.375592  1.662350   (9, 1, 5, 25)   
4        ...     -0.074181   1.702034  1.778492  (19, 2, 2, 17)   
5        ...      0.024267   1.185572  0.543540  (19, 0, 0, 21)   
...      ...           ...        ...       ...             ...   
35       ...      0.262422   1.114098  0.298874  (26, 1, 1, 12)   
78       ...     -0.186935   0.750983 -0.290204  (11, 2, 1, 26)   
116      ...     -0.069742   2.760162  0.433208  (19, 3, 1, 17)   
139      ...     -0.016959   0.972890 -0.092910  (15, 2, 3, 20)   
181      ...      0.324391   0.846292 -0.186705  (14, 4, 1, 21)   

                                           EF_rates EF_dprime  EF_metad  \
Subject                                                                   
1        (0.7894736842105263, 0.047619047619047616)  2.244885  2.540650   
2        (0.9705882352941176, 0.021739130434782608)  3.951340  2.355

In [51]:
getTotalTrialsAnalyzed(data=data, nRatings=11)

/opt/anaconda3/lib/python3.8/site-packages/metadPy/mle.py:738: RuntimeWarning: divide by zero encountered in double_scalars
  guess.extend(list(t2c1 - (meta_d1 * (t1c1 / d1))))
/opt/anaconda3/lib/python3.8/site-packages/metadPy/mle.py:738: RuntimeWarning: invalid value encountered in double_scalars
  guess.extend(list(t2c1 - (meta_d1 * (t1c1 / d1))))


Completed: VP
Completed: SM
Completed: EM
Completed: EF
Number of subjects: 717
Number of recorded trials: 28960
Number of analyzed trials: 28680
Number of non-analyzed trials: 280
Subjects not analyzed (@ domain): ['Subject 35 @ VP', 'Subject 78 @ VP', 'Subject 116 @ VP', 'Subject 139 @ VP', 'Subject 181 @ VP', 'Subject 109 @ SM', 'Subject 167 @ SM']
Percentage accounted for: 99.03314917127072%
Difference (%): 0.9668508287292821%


### `Arbuzova_unpub_3`

In [9]:
# read in data
data = pd.read_csv("../../Confidence Database/data_Arbuzova_unpub_3.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Task'])
data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses"})

# drop all NaNs
data = data.dropna().reset_index(drop=True)

# change dataset's 2s->1s and 1s->0s bc metadpy works with 0s and 1s
data['Stimuli'] = data['Stimuli'].map({1:0, 2:1})
data['Responses'] = data['Responses'].map({1:0, 2:1})

# create an accuracy column bc it is easier to work with
data['Accuracy'] = np.where((data['Stimuli'] == data['Responses']), 1, 0)

# get data in order by subject
data = data.sort_values(by=['Subject']).reset_index(drop=True)

In [10]:
getGroupData(data=data, nRatings=4, export='Arbuzova_unpub_3')

Memory
------------
Scores: (59, 15, 33, 79)
Rates: (0.7972972972972973, 0.29464285714285715)
Criterion: -0.14606742890770574
DPrime: 1.3526506384217538
MetaD: 2.059890405783755
MRatio: 1.5228547174510592
MDiff: 0.707239767362001
Done: Subject 1
Scores: (55, 27, 28, 71)
Rates: (0.6707317073170732, 0.2828282828282828)
Criterion: 0.06626273339218972
DPrime: 1.0043080488046785
MetaD: 1.4303789507576303
MRatio: 1.4242432413641002
MDiff: 0.42607090195295183
Done: Subject 2
Scores: (66, 23, 12, 80)
Rates: (0.7415730337078652, 0.13043478260869565)
Criterion: 0.2380678251234828
DPrime: 1.7457141818482085
MetaD: 2.1143822806374133
MRatio: 1.2111846845391905
MDiff: 0.3686680987892048
Done: Subject 3
Scores: (81, 17, 16, 75)
Rates: (0.826530612244898, 0.17582417582417584)
Criterion: -0.004573426190336405
DPrime: 1.8456070260201538
MetaD: 1.7132775115239431
MRatio: 0.9283002759360076
MDiff: -0.13232951449621067
Done: Subject 4
Scores: (67, 34, 23, 67)
Rates: (0.6633663366336634, 0.2555555555555555

Criterion: 0.03622314173027058
DPrime: 2.0060682790765334
MetaD: 2.7626779955152827
MRatio: 1.3771605006321341
MDiff: 0.7566097164387493
Done: Subject 38
Scores: (79, 17, 14, 88)
Rates: (0.8229166666666666, 0.13725490196078433)
Criterion: 0.0830991147353452
DPrime: 1.9906309960723572
MetaD: 2.6218478772527902
MRatio: 1.3170938674349313
MDiff: 0.6312168811804331
Done: Subject 39
Scores: (63, 21, 10, 91)
Rates: (0.75, 0.09900990099009901)
Criterion: 0.30636199131062386
DPrime: 1.9302321489088639
MetaD: 1.943168458143828
MRatio: 1.0067019447594823
MDiff: 0.012936309234964138
Done: Subject 40
Scores: (77, 25, 23, 65)
Rates: (0.7549019607843137, 0.26136363636363635)
Criterion: -0.025424970131155133
DPrime: 1.3130720956602933
MetaD: 1.1181070274450589
MRatio: 0.8515199059826232
MDiff: -0.1949650682152344
Done: Subject 41
------------
Completed: Memory
------------
Visual
------------
Scores: (66, 29, 33, 72)
Rates: (0.6947368421052632, 0.3142857142857143)
Criterion: -0.012791883208518595
DPr

Scores: (65, 30, 22, 63)
Rates: (0.6842105263157895, 0.25882352941176473)
Criterion: 0.08373549597651075
DPrime: 1.1124514554961917
MetaD: 0.8290815295485214
MRatio: 0.7452743447386857
MDiff: -0.2833699259476703
Done: Subject 36
Scores: (79, 31, 24, 46)
Rates: (0.7181818181818181, 0.34285714285714286)
Criterion: -0.08638541273142669
DPrime: 0.9703014603116995
MetaD: 1.4561613569046896
MRatio: 1.5007308722764496
MDiff: 0.4858598965929901
Done: Subject 37
Scores: (58, 33, 24, 65)
Rates: (0.6373626373626373, 0.2696629213483146)
Criterion: 0.13120734627119776
DPrime: 0.9535457735527921
MetaD: 0.8690891508128847
MRatio: 0.9114288741218656
MDiff: -0.0844566227399074
Done: Subject 38
Scores: (62, 29, 21, 68)
Rates: (0.6813186813186813, 0.23595505617977527)
Criterion: 0.12399259174997443
DPrime: 1.1757571300322398
MetaD: 0.10304809192553335
MRatio: 0.08764402893538713
MDiff: -1.0727090381067064
Done: Subject 40
Scores: (54, 31, 22, 73)
Rates: (0.6352941176470588, 0.23157894736842105)
Criterion

Scores: (65, 28, 26, 73)
Rates: (0.6989247311827957, 0.26262626262626265)
Criterion: 0.0569796408339886
DPrime: 1.1432493661470833
MetaD: 1.553903256256453
MRatio: 1.3591988784505808
MDiff: 0.4106538901093697
Done: Subject 37
Scores: (84, 21, 31, 56)
Rates: (0.8, 0.3563218390804598)
Criterion: -0.23665667723975187
DPrime: 1.1954862297766018
MetaD: 0.7985605881786297
MRatio: 0.6679797460551722
MDiff: -0.39692564159797206
Done: Subject 38
Scores: (72, 27, 28, 65)
Rates: (0.7272727272727273, 0.3010752688172043)
Criterion: -0.041637459187309045
DPrime: 1.1130034777519922
MetaD: 0.04661933729963484
MRatio: 0.0418860661547932
MDiff: -1.0663841404523573
Done: Subject 39
Scores: (65, 29, 26, 72)
Rates: (0.6914893617021277, 0.2653061224489796)
Criterion: 0.0634976393567655
DPrime: 1.114213619683281
MetaD: 0.12145222344094814
MRatio: 0.10900263764095017
MDiff: -0.992761396242333
Done: Subject 40
Scores: (46, 52, 64, 30)
Rates: (0.46938775510204084, 0.6808510638297872)
Criterion: -0.1966355052896

Scores: (75, 24, 31, 62)
Rates: (0.7575757575757576, 0.3333333333333333)
Criterion: -0.13389920552480125
DPrime: 1.116164840773377
MetaD: 0.973904653004986
MRatio: 0.8725455393579493
MDiff: -0.1422601877683911
Done: Subject 37
Scores: (66, 21, 31, 74)
Rates: (0.7586206896551724, 0.29523809523809524)
Criterion: -0.08186324945888657
DPrime: 1.2250372694626959
MetaD: 1.4732170923186063
MRatio: 1.2025896101632587
MDiff: 0.24817982285591045
Done: Subject 38
Scores: (66, 27, 25, 74)
Rates: (0.7096774193548387, 0.25252525252525254)
Criterion: 0.057060837275564824
DPrime: 1.204778854298124
MetaD: 0.5568083527258681
MRatio: 0.46216643887749154
MDiff: -0.6479705015722558
Done: Subject 39
Scores: (64, 34, 21, 73)
Rates: (0.6530612244897959, 0.22340425531914893)
Criterion: 0.1835740289871687
DPrime: 1.1404493199687034
MetaD: -0.32994079974012613
MRatio: -0.28930772631718565
MDiff: -1.4703901197088296
Done: Subject 40
Scores: (34, 60, 77, 21)
Rates: (0.3617021276595745, 0.7857142857142857)
Criterio

FileNotFoundError: [Errno 2] No such file or directory: '../exports/analysis_Arbuzova_unpub_3.csv'

In [49]:
getTotalTrialsAnalyzed(data=data, nRatings=4)

Completed: memory
Completed: visual
Completed: motor
Completed: visuomotor
Number of subjects: 150
Number of recorded trials: 28639
Number of analyzed trials: 28099
Number of non-analyzed trials: 540
Subjects not analyzed (@ domain): ['Subject 5 @ visual', 'Subject 29 @ visual', 'Subject 39 @ visual']
Percentage accounted for: 98.11445930374664%
Difference (%): 1.8855406962533627%


### `Sadeghi_2017`

In [46]:
# read in data
data = pd.read_csv("../Confidence Database/data_Sadeghi_2017_memory.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'group'])

data2 = pd.read_csv("../Confidence Database/data_Sadeghi_2017_perception.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'group'])

data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses", "group": "Task"})

data2 = data2.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses", "group": "Task"})


# get only control patients
data = data.loc[data['Task'] == 'control']
data2 = data2.loc[data2['Task'] == 'control']

# drop all NaNs
data = data.dropna().reset_index(drop=True)
data2 = data2.dropna().reset_index(drop=True)

# change task values according to the dataset
data['Task'] = 'memory'
data2['Task'] = 'perception'

# change perception dataset's 2s->1s and 1s->0s bc metadpy works with 0s and 1s
data2['Stimuli'] = data2['Stimuli'].map({1:0, 2:1})
data2['Responses'] = data2['Responses'].map({1:0, 2:1})

# merge all data into just 'data' dataframe
data = data.append(data2)

In [8]:
getGroupData(data=data, nRatings=6, export='Sadeghi_2017')

Memory
------------
Done: Subject 503
Done: Subject 504
Done: Subject 507
Done: Subject 508
Done: Subject 509
Done: Subject 510
Done: Subject 511
Done: Subject 512
Done: Subject 516
Done: Subject 517
Done: Subject 518
Done: Subject 519
Done: Subject 520
Done: Subject 521
Done: Subject 522
Done: Subject 523
Done: Subject 524
Done: Subject 527
Done: Subject 528
Done: Subject 530
Done: Subject 531
Done: Subject 532
Done: Subject 533
------------
Completed: Memory
------------
Perception
------------
Done: Subject 501
Done: Subject 503
Done: Subject 504
Done: Subject 505
Done: Subject 507
Done: Subject 508
Done: Subject 509
Done: Subject 510
Done: Subject 511
Done: Subject 512
Done: Subject 516
Done: Subject 517
Done: Subject 518
Done: Subject 519
Done: Subject 520
Done: Subject 521
Done: Subject 522
Done: Subject 523
Done: Subject 524
Done: Subject 527
Done: Subject 528
Done: Subject 530
Done: Subject 531
Done: Subject 532
Done: Subject 533
------------
Completed: Perception
------------


memory_scores  memory_rates  memory_dprime  memory_metad  \
Subject                                                                
503      (46, 54, 14, 86)  (0.46, 0.14)       0.965006      1.218505   
504      (70, 30, 19, 81)   (0.7, 0.19)       1.385355      2.213079   
507      (74, 26, 43, 57)  (0.74, 0.43)       0.810647      0.845147   
508      (44, 56, 21, 79)  (0.44, 0.21)       0.647035      0.907154   
509       (92, 8, 37, 63)  (0.92, 0.37)       1.706079      0.865935   
510      (65, 35, 19, 81)  (0.65, 0.19)       1.247955      0.976878   
511      (84, 16, 33, 67)  (0.84, 0.33)       1.415986      0.897530   
512      (81, 19, 31, 69)  (0.81, 0.31)       1.357166      0.930412   
516      (87, 13, 23, 77)  (0.87, 0.23)       1.839311      1.311975   
517      (59, 41, 20, 80)   (0.59, 0.2)       1.056312      0.479784   
518      (64, 36, 18, 82)  (0.64, 0.18)       1.258113      1.359091   
519      (71, 29, 48, 52)  (0.71, 0.48)       0.596977      0.800033   
520       (92, 8, 28, 72)  (0.92, 0.28)       1.954015      2.437659   
521      (62, 38, 23, 77)  (0.62, 0.23)       1.032433      0.583035   
522       (91, 9, 32, 68)  (0.91, 0.32)       1.778886     -0.516571   
523      (77, 23, 45, 55)  (0.77, 0.45)       0.854482      0.597467   
524      (78, 22, 19, 81)  (0.78, 0.19)       1.629505      0.811403   
527      (61, 39, 21, 79)  (0.61, 0.21)       1.072980      0.799329   
528      (65, 35, 28, 72)  (0.65, 0.28)       0.957697      0.777074   
530      (82, 18, 43, 57)  (0.82, 0.43)       1.077967      1.197528   
531      (51, 49, 12, 88)  (0.51, 0.12)       1.181203      1.073834   
532      (68, 32, 21, 79)  (0.68, 0.21)       1.259219      0.915391   
533      (66, 34, 22, 78)  (0.66, 0.22)       1.171007      1.048803   
501                   NaN           NaN            NaN           NaN   
505                   NaN           NaN            NaN           NaN   

         memory_criterion  memory_mratio  memory_mdiff perception_scores  \
Subject                                                                    
503              0.590377       1.262692      0.253500  (66, 24, 27, 63)   
504              0.176748       1.597481      0.827724  (69, 21, 33, 57)   
507             -0.233486       1.042559      0.034501  (48, 42, 13, 77)   
508              0.478695       1.402018      0.260120   (43, 47, 5, 85)   
509             -0.536609       0.507559     -0.840144  (68, 22, 29, 61)   
510              0.246288       0.782783     -0.271077  (56, 34, 15, 75)   
511             -0.277272       0.633855     -0.518457   (45, 45, 6, 84)   
512             -0.191023       0.685555     -0.426754  (56, 34, 22, 68)   
516             -0.193772       0.713297     -0.527337  (51, 39, 12, 78)   
517              0.307038       0.454206     -0.576528  (58, 32, 18, 72)   
518              0.278453       1.080261      0.100978  (63, 27, 27, 63)   
519             -0.251616       1.340140      0.203056  (64, 26, 26, 64)   
520             -0.411115       1.247513      0.483644  (59, 31, 20, 70)   
521              0.216683       0.564720     -0.449398  (61, 29, 20, 70)   
522             -0.436528      -0.290390     -2.295457  (59, 31, 21, 69)   
523             -0.306593       0.699216     -0.257015  (52, 38, 19, 71)   
524              0.052852       0.497945     -0.818102  (57, 33, 20, 70)   
527              0.263551       0.744962     -0.273651  (63, 27, 25, 65)   
528              0.098761       0.811399     -0.180623  (49, 41, 14, 76)   
530             -0.369495       1.110914      0.119562  (73, 17, 37, 53)   
531              0.574959       0.909102     -0.107369  (60, 30, 21, 69)   
532              0.169361       0.726951     -0.343828  (58, 32, 19, 71)   
533              0.179865       0.895642     -0.122204  (48, 42, 11, 79)   
501                   NaN            NaN           NaN  (55, 35, 15, 75)   
505                   NaN            NaN           NaN  (70, 20, 31, 59)   

              

In [47]:
getTotalTrialsAnalyzed(data=data, nRatings=6)

Completed: memory
Completed: perception
Number of subjects: 48
Number of recorded trials: 9500
Number of analyzed trials: 9100
Number of non-analyzed trials: 400
Subjects not analyzed (@ domain): ['Subject 501 @ memory', 'Subject 505 @ memory']
Percentage accounted for: 95.78947368421052%
Difference (%): 4.210526315789476%


### `Samaha_2016`

In [44]:
# read in data
data = pd.read_csv("../Confidence Database/data_Samaha_2016.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Task', 'Condition', 'Accuracy'])
data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses"})

# drop all NaNs
data = data.dropna().reset_index(drop=True)

# change dataset's -45s->0s and 45s->1s bc metadpy works with 0s and 1s
data['Stimuli'] = data['Stimuli'].map({-45:0, 45:1, 0:0, 1:1})
data['Responses'] = data['Responses'].map({-45:0, 45:1, 0:0, 1:1})

# rename tasks based on conditions - 0.5 -> low; 1 -> high
data['Task'] = np.where(data['Task'].eq("percept") & data['Condition'].eq(0.5), 'percept_low', data['Task'])
data['Task'] = np.where(data['Task'].eq("percept") & data['Condition'].eq(1), 'percept_high', data['Task'])
data['Task'] = np.where(data['Task'].eq("wm") & data['Condition'].eq(0.5), 'wm_low', data['Task'])
data['Task'] = np.where(data['Task'].eq("wm") & data['Condition'].eq(1), 'wm_high', data['Task'])

In [10]:
getGroupData(data=data, nRatings=4, export='Samaha_2016')

Percept_low
------------
Done: Subject 1
Done: Subject 2
Done: Subject 3
Done: Subject 4
Done: Subject 5
Done: Subject 6
Done: Subject 7
Done: Subject 8
Done: Subject 9
Done: Subject 10
Done: Subject 11
Done: Subject 12
Done: Subject 13
Done: Subject 14
Done: Subject 15
------------
Completed: Percept_low
------------
Percept_high
------------
Done: Subject 1
Done: Subject 3
Done: Subject 4
Done: Subject 5
Done: Subject 6
Done: Subject 7
Done: Subject 8
Done: Subject 9
Done: Subject 10
Done: Subject 11
Done: Subject 12
Done: Subject 13
Done: Subject 14
Done: Subject 15
------------
Completed: Percept_high
------------
Wm_high
------------
Done: Subject 1
Done: Subject 2
Done: Subject 3
Done: Subject 4
Done: Subject 5
Done: Subject 6
Done: Subject 7
Done: Subject 8
Done: Subject 9
Done: Subject 10
Done: Subject 11
Done: Subject 12
Done: Subject 13
Done: Subject 14
Done: Subject 15
------------
Completed: Wm_high
------------
Wm_low
------------
Done: Subject 1
Done: Subject 2
Done: Subj

percept_low_scores                          percept_low_rates  \
Subject                                                                 
1         (60, 17, 30, 56)   (0.7792207792207793, 0.3488372093023256)   
2         (35, 47, 20, 56)   (0.4268292682926829, 0.2631578947368421)   
3         (38, 32, 21, 60)  (0.5428571428571428, 0.25925925925925924)   
4          (72, 12, 7, 71)  (0.8571428571428571, 0.08974358974358974)   
5         (47, 29, 26, 63)   (0.618421052631579, 0.29213483146067415)   
6         (63, 13, 21, 53)  (0.8289473684210527, 0.28378378378378377)   
7         (66, 22, 33, 50)                (0.75, 0.39759036144578314)   
8         (63, 25, 22, 52)   (0.7159090909090909, 0.2972972972972973)   
9         (65, 15, 14, 70)              (0.8125, 0.16666666666666666)   
10        (61, 25, 28, 41)   (0.7093023255813954, 0.4057971014492754)   
11        (48, 31, 18, 61)  (0.6075949367088608, 0.22784810126582278)   
12        (49, 29, 28, 58)  (0.6282051282051282, 0.32558139534883723)   
13        (55, 27, 21, 59)               (0.6707317073170732, 0.2625)   
14        (54, 16, 17, 69)  (0.7714285714285715, 0.19767441860465115)   
15        (61, 17, 18, 59)   (0.782051282051282, 0.23376623376623376)   

         percept_low_dprime  percept_low_metad  percept_low_criterion  \
Subject                                                                 
1                  1.161780           0.950693              -0.190551   
2                  0.442039          -0.045524               0.409046   
3                  0.742705           1.270229               0.268998   
4                  2.432126           1.281675               0.137383   
5                  0.860894           0.606719               0.122911   
6                  1.487578           0.984685              -0.189188   
7                  0.922106           0.681782              -0.207450   
8                  1.105855           0.777096              -0.019271   
9                  1.824746           1.097801               0.040138   
10                 0.769361           1.019710              -0.156489   
11                 1.004295           0.592897               0.236448   
12                 0.791164           1.509278               0.062522   
13                 1.062944           0.988806               0.096861   
14                 1.557679           0.180503               0.053199   
15                 1.472959           0.060764              -0.026320   

         percept_low_mratio  percept_low_mdiff percept_high_scores  \
Subject                                                              
1                  0.818307          -0.211087    (49, 31, 24, 53)   
2                 -0.102988          -0.487563                 NaN   
3                  1.710273           0.527524    (49, 34, 20, 66)   
4                  0.526977          -1.150450    (69, 10, 10, 69)   
5                  0.704755          -0.254175    (54, 30, 20, 51)   
6                  0.661938          -0.502893    (76, 13, 17, 64)   
7                  0.739375          -0.240324    (59, 18, 15, 57)   
8                  0.702710          -0.328759    (47, 24, 29, 58)   
9                  0.601619          -0.726945    (53, 27, 15, 61)   
10                 1.325399           0.250350    (68, 14, 20, 63)   
11                 0.590362          -0.411397    (45, 33, 10, 74)   
12                 1.907667           0.718114    (55, 20, 27, 54)   
13                 0.930253          -0.074138    (49, 25, 16, 68)   
14                 0.115879          -1.377176    (71, 15, 22, 56)   
15                 0.041253          -1.412195    (69, 16, 22, 58)   

                                percept_high_rates  percept_high_dprime  ...  \
Subject                                                                  ...   
1                     (0.6125, 0.3116883116883117)             0.766471  ...   
2                                              NaN                  NaN  ...   
3        (0.5

In [45]:
getTotalTrialsAnalyzed(data=data, nRatings=4)

Completed: percept_low
Completed: percept_high
Completed: wm_high
Completed: wm_low
Number of subjects: 59
Number of recorded trials: 8400
Number of analyzed trials: 8238
Number of non-analyzed trials: 162
Subjects not analyzed (@ domain): ['Subject 2 @ percept_high']
Percentage accounted for: 98.07142857142857%
Difference (%): 1.928571428571424%


### `Samaha_2017_exp3`

In [42]:
# read in data
data = pd.read_csv("../Confidence Database/data_Samaha_2017_exp3.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Task'])
data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses"})

# drop all NaNs
data = data.dropna().reset_index(drop=True)

# change dataset's values bc metadpy works with 0s and 1s
data['Stimuli'] = data['Stimuli'].map({-1:0, 0:0, 1:1})
data['Responses'] = data['Responses'].map({-1:0, 0:0, 1:1})

# create an accuracy column bc it is easier to work with
data['Accuracy'] = np.where((data['Stimuli'] == data['Responses']), 1, 0)

# get data in order by subject
data = data.sort_values(by=['Subject'])
data = data.reset_index(drop=True)

In [12]:
getGroupData(data=data, nRatings=4, export='Samaha_2017_exp3')

Oriperc
------------
Done: Subject 1
Done: Subject 2
Done: Subject 3
Done: Subject 4
Done: Subject 5
Done: Subject 6
Done: Subject 7
Done: Subject 8
Done: Subject 9
Done: Subject 10
Done: Subject 11
Done: Subject 12
Done: Subject 13
Done: Subject 14
Done: Subject 15
Done: Subject 16
Done: Subject 17
Done: Subject 18
Done: Subject 19
Done: Subject 20
------------
Completed: Oriperc
------------
Contrast
------------
Done: Subject 1
Done: Subject 2
Done: Subject 3
Done: Subject 4
Done: Subject 5
Done: Subject 6
Done: Subject 7
Done: Subject 8
Done: Subject 9
Done: Subject 10
Done: Subject 11
Done: Subject 12
Done: Subject 13
Done: Subject 14
Done: Subject 15
Done: Subject 16
Done: Subject 17
Done: Subject 18
Done: Subject 19
Done: Subject 20
------------
Completed: Contrast
------------
Oriwm
------------
Done: Subject 1
Done: Subject 2
Done: Subject 3
Done: Subject 4
Done: Subject 5
Done: Subject 6
Done: Subject 7
Done: Subject 8
Done: Subject 9
Done: Subject 10
Done: Subject 11
Done: S

oriperc_scores                               oriperc_rates  \
Subject                                                                   
1        (122, 37, 29, 112)   (0.7672955974842768, 0.20567375886524822)   
2        (120, 24, 39, 117)                  (0.8333333333333334, 0.25)   
3        (113, 39, 16, 132)    (0.743421052631579, 0.10810810810810811)   
4        (105, 45, 34, 116)                  (0.7, 0.22666666666666666)   
5        (137, 18, 44, 101)   (0.8838709677419355, 0.30344827586206896)   
6        (136, 16, 21, 127)   (0.8947368421052632, 0.14189189189189189)   
7         (112, 41, 4, 143)  (0.7320261437908496, 0.027210884353741496)   
8        (108, 28, 27, 137)   (0.7941176470588235, 0.16463414634146342)   
9        (125, 35, 36, 104)               (0.78125, 0.2571428571428571)   
10       (121, 37, 22, 120)   (0.7658227848101266, 0.15492957746478872)   
11       (129, 17, 52, 102)   (0.8835616438356164, 0.33766233766233766)   
12        (85, 58, 26, 131)   (0.5944055944055944, 0.16560509554140126)   
13        (149, 19, 36, 96)    (0.8869047619047619, 0.2727272727272727)   
14       (127, 29, 21, 123)   (0.8141025641025641, 0.14583333333333334)   
15       (112, 30, 25, 133)   (0.7887323943661971, 0.15822784810126583)   
16       (114, 25, 24, 137)   (0.8201438848920863, 0.14906832298136646)   
17        (126, 28, 9, 137)   (0.8181818181818182, 0.06164383561643835)   
18        (144, 11, 53, 92)   (0.9290322580645162, 0.36551724137931035)   
19       (124, 21, 44, 111)    (0.8551724137931035, 0.2838709677419355)   
20       (118, 24, 32, 126)   (0.8309859154929577, 0.20253164556962025)   

         oriperc_dprime  oriperc_metad  oriperc_criterion  oriperc_mratio  \
Subject                                                                     
1              1.538780       0.511248           0.045778        0.332242   
2              1.627749       0.509163          -0.146466        0.312802   
3              1.871609       0.878189           0.291362        0.469216   
4              1.264467       0.272997           0.112734        0.215899   
5              1.692726      -0.741136          -0.340027       -0.437836   
6              2.299299       1.753698          -0.090131        0.762710   
7              2.489253       1.981487           0.652256        0.796016   
8              1.780700       1.749291           0.077398        0.982362   
9              1.417364       1.455795          -0.062121        1.027115   
10             1.725158       0.279488           0.145179        0.162007   
11             1.595748       1.111930          -0.387065        0.696808   
12             1.200410       0.036644           0.366393        0.030526   
13             1.797831       0.851142          -0.302823        0.473427   
14             1.929525       0.526130           0.080678        0.272674   
15             1.787977       1.206600           0.099869        0.674841   
16             1.938382       1.211756           0.062262        0.625138   
17             2.417770       1.092208           0.316331        0.451742   
18             1.789949       0.032695          -0.562436        0.018266   
19             1.615553       1.212808          -0.243749        0.750708   
20             1.774926       1.216967          -0.062728        0.685644   

         oriperc_mdiff     contrast_scores  \
Subject                                      
1            -1.027532  (122, 27, 33, 118)   
2            -1.118585  (116, 29, 34, 121)   
3            -0.993420  (119, 19, 43, 119)   
4            -0.991470      (46, 4, 9, 41)   
5            -2.433862    (97, 60, 7, 136)   
6            -0.545600  (102, 30, 27, 141)   
7            -0.507767  (124, 24, 36, 116)   
8            -0.031409   (83, 49, 14, 154)   
9             0.038432  (138, 21, 40, 101)   
10           -1.445670  (117, 41, 21, 121)   
11           -0.483819  (129, 32, 27, 112)   
12           -1.163766    (154, 5, 54, 87)   
13           -0.946689 

In [43]:
getTotalTrialsAnalyzed(data=data, nRatings=4)

Completed: oriperc
Completed: contrast
Completed: oriwm
Number of subjects: 60
Number of recorded trials: 17700
Number of analyzed trials: 17700
Number of non-analyzed trials: 0
Subjects not analyzed (@ domain): []
Percentage accounted for: 100.0%
Difference (%): 0.0%


### `Schmidt_2019`

In [40]:
# read in data
data = pd.read_csv("../Confidence Database/data_Schmidt_2019_memory.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Accuracy', 'Condition'])

data2 = pd.read_csv("../Confidence Database/data_Schmidt_2019_perception.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Accuracy', 'Condition'])

data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses",
                            "Condition": "Task"})

data2 = data2.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses",
                              "Condition": "Task"})

# get only first 200 trials for memory bc they are 'pre-training'
data = data.groupby('Subject').head(200)

# get only first 300 trials for perception bc they are 'pre-training'
data2 = data2.groupby('Subject').head(300)

# drop all NaNs
data = data.dropna().reset_index(drop=True)
data2 = data2.dropna().reset_index(drop=True)

# change task values according to the dataset
data['Task'] = 'memory'
data2['Task'] = 'perception'

# change dataset's 2s->1s and 1s->0s bc metadpy works with 0s and 1s
data['Stimuli'] = data['Stimuli'].map({1:0, 2:1})
data['Responses'] = data['Responses'].map({1:0, 2:1})

data2['Stimuli'] = data2['Stimuli'].map({1:0, 2:1})
data2['Responses'] = data2['Responses'].map({1:0, 2:1})

# merge all data into just 'data' dataframe
data = data.append(data2)

In [14]:
getGroupData(data=data, nRatings=4, export='Schmidt_2019')

Memory
------------
Done: Subject 1


/opt/anaconda3/lib/python3.8/site-packages/metadPy/mle.py:738: RuntimeWarning: invalid value encountered in double_scalars
  guess.extend(list(t2c1 - (meta_d1 * (t1c1 / d1))))


Done: Subject 15
Done: Subject 22
Done: Subject 29
Done: Subject 30


/opt/anaconda3/lib/python3.8/site-packages/metadPy/mle.py:738: RuntimeWarning: invalid value encountered in double_scalars
  guess.extend(list(t2c1 - (meta_d1 * (t1c1 / d1))))


Done: Subject 42
Done: Subject 43
Done: Subject 55
Done: Subject 56
Done: Subject 81
Done: Subject 82
Done: Subject 83
Done: Subject 85
Done: Subject 86
Done: Subject 87
Done: Subject 90
Done: Subject 91
Done: Subject 92
Done: Subject 94
Done: Subject 97
------------
Completed: Memory
------------
Perception
------------
Done: Subject 2
Done: Subject 5
Done: Subject 15
Done: Subject 22
Done: Subject 28
Done: Subject 29
Done: Subject 31
Done: Subject 42
Done: Subject 43
Done: Subject 53
Done: Subject 55


/opt/anaconda3/lib/python3.8/site-packages/metadPy/mle.py:738: RuntimeWarning: divide by zero encountered in double_scalars
  guess.extend(list(t2c1 - (meta_d1 * (t1c1 / d1))))
/opt/anaconda3/lib/python3.8/site-packages/metadPy/mle.py:738: RuntimeWarning: invalid value encountered in double_scalars
  guess.extend(list(t2c1 - (meta_d1 * (t1c1 / d1))))


Done: Subject 81
Done: Subject 82
Done: Subject 83
Done: Subject 85
Done: Subject 86
Done: Subject 87
Done: Subject 90
Done: Subject 91


/opt/anaconda3/lib/python3.8/site-packages/metadPy/mle.py:738: RuntimeWarning: invalid value encountered in double_scalars
  guess.extend(list(t2c1 - (meta_d1 * (t1c1 / d1))))


Done: Subject 94
Done: Subject 95
Done: Subject 97
------------
Completed: Perception
------------
Exported: 'analysis_Schmidt_2019.csv'


memory_scores                               memory_rates  \
Subject                                                                
1        (74, 20, 19, 87)   (0.7872340425531915, 0.1792452830188679)   
15       (85, 18, 11, 86)   (0.8252427184466019, 0.1134020618556701)   
22       (71, 40, 25, 64)   (0.6396396396396397, 0.2808988764044944)   
29       (58, 42, 10, 90)                                (0.58, 0.1)   
30        (96, 22, 9, 73)  (0.8135593220338984, 0.10975609756097561)   
42       (74, 25, 13, 88)  (0.7474747474747475, 0.12871287128712872)   
43       (90, 18, 14, 78)  (0.8333333333333334, 0.15217391304347827)   
55       (79, 26, 16, 79)  (0.7523809523809524, 0.16842105263157894)   
56       (64, 25, 14, 97)  (0.7191011235955056, 0.12612612612612611)   
81       (72, 30, 20, 78)  (0.7058823529411765, 0.20408163265306123)   
82       (84, 18, 16, 82)  (0.8235294117647058, 0.16326530612244897)   
83       (72, 29, 17, 82)   (0.7128712871287128, 0.1717171717171717)   
85       (68, 34, 26, 72)   (0.6666666666666666, 0.2653061224489796)   
86       (67, 33, 26, 74)                               (0.67, 0.26)   
87       (74, 19, 7, 100)  (0.7956989247311828, 0.06542056074766354)   
90       (62, 37, 24, 77)   (0.6262626262626263, 0.2376237623762376)   
91       (81, 26, 27, 66)   (0.7570093457943925, 0.2903225806451613)   
92       (80, 16, 33, 71)   (0.8333333333333334, 0.3173076923076923)   
94       (70, 29, 29, 72)   (0.7070707070707071, 0.2871287128712871)   
97       (68, 29, 27, 76)   (0.7010309278350515, 0.2621359223300971)   
2                     NaN                                        NaN   
5                     NaN                                        NaN   
28                    NaN                                        NaN   
31                    NaN                                        NaN   
53                    NaN                                        NaN   
95                    NaN                                        NaN   

         memory_dprime  memory_metad  memory_criterion  memory_mratio  \
Subject                                                                 
1            -0.890507     -0.098523          0.060692       0.110637   
15            1.098836      1.362886          0.136551       1.240299   
22            1.824839     -0.016143          0.111339      -0.008846   
29            1.348980      1.004837          0.539829       0.744887   
30            0.821131     -0.487097          0.168369      -0.593203   
42            1.119273      1.047802          0.232966       0.936146   
43            0.541202      0.469818          0.029866       0.868101   
55            1.468989      1.428493          0.139211       0.972433   
56            0.967422     -1.133427          0.282361      -1.171595   
81            0.534723     -0.274742          0.142868      -0.513803   
82            1.110278     -0.085926          0.026113      -0.077392   
83            0.529068      1.281108          0.192804       2.421444   
85            1.360778      2.022806          0.098172       1.486508   
86            1.382994     -0.473837          0.101716      -0.342617   
87           -0.223349      0.248369          0.342219      -1.112020   
90            0.125661      0.855845          0.195998       6.810725   
91           -0.318639      0.438581         -0.072136      -1.376418   
92           -0.318639      2.680164         -0.246090      -8.411276   
94           -0.648782     -0.565375          0.008473       0.871440   
97            0.674490      0.557967          0.054703       0.827243   
2                  NaN           NaN               NaN            NaN   
5                  NaN           NaN               NaN            NaN   
28                 NaN           NaN               NaN            NaN   
31                 NaN           NaN               NaN            NaN   
53                 NaN           NaN               NaN            NaN   
95                 NaN  

In [41]:
getTotalTrialsAnalyzed(data=data, nRatings=4)

/opt/anaconda3/lib/python3.8/site-packages/metadPy/mle.py:738: RuntimeWarning: invalid value encountered in double_scalars
  guess.extend(list(t2c1 - (meta_d1 * (t1c1 / d1))))
/opt/anaconda3/lib/python3.8/site-packages/metadPy/mle.py:738: RuntimeWarning: invalid value encountered in double_scalars
  guess.extend(list(t2c1 - (meta_d1 * (t1c1 / d1))))


Completed: memory


/opt/anaconda3/lib/python3.8/site-packages/metadPy/mle.py:738: RuntimeWarning: divide by zero encountered in double_scalars
  guess.extend(list(t2c1 - (meta_d1 * (t1c1 / d1))))
/opt/anaconda3/lib/python3.8/site-packages/metadPy/mle.py:738: RuntimeWarning: invalid value encountered in double_scalars
  guess.extend(list(t2c1 - (meta_d1 * (t1c1 / d1))))
/opt/anaconda3/lib/python3.8/site-packages/metadPy/mle.py:738: RuntimeWarning: invalid value encountered in double_scalars
  guess.extend(list(t2c1 - (meta_d1 * (t1c1 / d1))))


Completed: perception
Number of subjects: 42
Number of recorded trials: 13500
Number of analyzed trials: 10600
Number of non-analyzed trials: 2900
Subjects not analyzed (@ domain): ['Subject 2 @ memory', 'Subject 5 @ memory', 'Subject 28 @ memory', 'Subject 31 @ memory', 'Subject 39 @ memory', 'Subject 53 @ memory', 'Subject 95 @ memory', 'Subject 1 @ perception', 'Subject 30 @ perception', 'Subject 39 @ perception', 'Subject 56 @ perception', 'Subject 92 @ perception']
Percentage accounted for: 78.51851851851852%
Difference (%): 21.48148148148148%


### `Skora_2016`

In [38]:
# read in data
data = pd.read_csv("../Confidence Database/data_Skora_2016.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Condition', 'Accuracy'])
data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses", "Condition": "Task"})

# get data in order by subject
data = data.sort_values(by=['Subject']).reset_index(drop=True)

In [16]:
getGroupData(data=data, nRatings=4, export='Skora_2016')

Fm
------------
Done: Subject 1
Done: Subject 3
Done: Subject 4
Done: Subject 5
Done: Subject 6
Done: Subject 7
Done: Subject 8
Done: Subject 9
Done: Subject 10
Done: Subject 11
Done: Subject 12
Done: Subject 13
Done: Subject 14
Done: Subject 15
Done: Subject 16
Done: Subject 17
Done: Subject 18
Done: Subject 19
------------
Completed: Fm
------------
Im
------------
Done: Subject 1
Done: Subject 2
Done: Subject 3
Done: Subject 4
Done: Subject 5
Done: Subject 6
Done: Subject 7
Done: Subject 8
Done: Subject 9
Done: Subject 10
Done: Subject 11
Done: Subject 12
Done: Subject 13
Done: Subject 14
Done: Subject 15
Done: Subject 16
Done: Subject 17
Done: Subject 18
Done: Subject 19
------------
Completed: Im
------------
Wm
------------
Done: Subject 1
Done: Subject 2
Done: Subject 3
Done: Subject 5
Done: Subject 6
Done: Subject 7
Done: Subject 8
Done: Subject 9
Done: Subject 10
Done: Subject 11
Done: Subject 12
Done: Subject 13
Done: Subject 14
Done: Subject 15
Done: Subject 16
Done: Subject

FM_scores                                   FM_rates  \
Subject                                                                
1        (46, 14, 12, 48)                  (0.7666666666666667, 0.2)   
3        (46, 14, 24, 36)                  (0.7666666666666667, 0.4)   
4        (49, 11, 13, 47)  (0.8166666666666667, 0.21666666666666667)   
5         (52, 8, 12, 48)                  (0.8666666666666667, 0.2)   
6         (54, 6, 26, 34)                 (0.9, 0.43333333333333335)   
7         (51, 9, 13, 47)                (0.85, 0.21666666666666667)   
8          (4, 56, 59, 1)  (0.06666666666666667, 0.9833333333333333)   
9        (48, 12, 12, 48)                                 (0.8, 0.2)   
10        (51, 9, 23, 37)                (0.85, 0.38333333333333336)   
11        (39, 21, 8, 52)                (0.65, 0.13333333333333333)   
12       (49, 11, 28, 32)   (0.8166666666666667, 0.4666666666666667)   
13       (45, 15, 29, 31)                (0.75, 0.48333333333333334)   
14        (30, 30, 6, 54)                                 (0.5, 0.1)   
15        (53, 7, 11, 49)  (0.8833333333333333, 0.18333333333333332)   
16       (50, 10, 12, 48)                  (0.8333333333333334, 0.2)   
17       (35, 25, 17, 43)   (0.5833333333333334, 0.2833333333333333)   
18       (39, 21, 16, 44)                (0.65, 0.26666666666666666)   
19       (49, 11, 12, 48)                  (0.8166666666666667, 0.2)   
2                     NaN                                        NaN   

         FM_dprime  FM_metad  FM_criterion  FM_mratio  FM_mdiff  \
Subject                                                           
1         1.537887  1.993427  5.685397e-02   1.296212  0.455540   
3         0.988682  1.884945 -2.372831e-01   1.906522  0.896262   
4         1.759226  1.837252 -5.961721e-02   1.044353  0.078026   
5         1.955041  2.373046 -1.345752e-01   1.213809  0.418005   
6         1.475422  1.407896 -5.568288e-01   0.954233 -0.067525   
7         1.813549  1.984212 -1.264665e-01   1.094105  0.170663   
8        -3.535945 -4.388311 -3.134796e-01   1.241058 -0.852367   
9         1.624151  0.351250 -5.551115e-17   0.216267 -1.272901   
10        1.358835  1.840500 -3.698478e-01   1.354469  0.481665   
11        1.503035  1.218943  3.627256e-01   0.810988 -0.284092   
12        0.965624  1.086292 -4.095415e-01   1.124965  0.120669   
13        0.724355  0.284744 -3.163502e-01   0.393100 -0.439611   
14        1.245046  1.511174  6.407758e-01   1.213749  0.266128   
15        2.043708  2.649676 -1.445407e-01   1.296504  0.605968   
16        1.769959  2.189271 -6.290017e-02   1.236905  0.419312   
17        0.804689  0.784604  1.812696e-01   0.975041 -0.020084   
18        1.011409  0.621388  1.188026e-01   0.614378 -0.390022   
19        1.707528  2.477351 -3.055678e-02   1.450840  0.769823   
2              NaN       NaN           NaN        NaN       NaN   

                IM_scores                                   IM_rates  \
Subject                                                                
1          (55, 5, 4, 56)  (0.9166666666666666, 0.06666666666666667)   
3         (46, 14, 6, 54)                  (0.7666666666666667, 0.1)   
4        (49, 11, 11, 49)  (0.8166666666666667, 0.18333333333333332)   
5          (56, 4, 2, 58)  (0.9333333333333333, 0.03333333333333333)   
6         (58, 2, 17, 43)   (0.9666666666666667, 0.2833333333333333)   
7          (54, 6, 3, 57)                                (0.9, 0.05)   
8          (3, 57, 60, 0)                 (0.05, 0.9916666666666667)   
9         (53, 7, 23, 37)  (0.8833333333333333, 0.38333333333333336)   
10        (40, 20, 9, 51)                 (0.6666666666666666, 0.15)   
11        (46, 14, 7, 53)  (0.7666666666666667, 0.11666666666666667)   
12         (51, 9, 7, 53)                (0.85, 0.11666666666666667)   
13       (45, 15, 23, 37)                (0.75, 0.38333333333333336)   
14         (52, 8, 4, 56)  (0.8666666666666667, 0.06666666666666667)   
15         (56,

In [39]:
getTotalTrialsAnalyzed(data=data, nRatings=4)

Completed: FM
Completed: IM
Completed: WM
Number of subjects: 54
Number of recorded trials: 6840
Number of analyzed trials: 6480
Number of non-analyzed trials: 360
Subjects not analyzed (@ domain): ['Subject 2 @ FM', 'Subject 4 @ WM', 'Subject 18 @ WM']
Percentage accounted for: 94.73684210526315%
Difference (%): 5.263157894736848%


### `Xu_2019_Exp2`

In [36]:
# read in data
data = pd.read_csv("../Confidence Database/data_Xu_2019_Exp2.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Task', 'Condition'])
data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses"})

# drop all NaNs
data = data.dropna().reset_index(drop=True)

# change dataset's 2s->1s and 1s->0s bc metadpy works with 0s and 1s
data['Stimuli'] = data['Stimuli'].map({1:0, 2:1})
data['Responses'] = data['Responses'].map({1:0, 2:1})

# create an accuracy column bc it is easier to work with
data['Accuracy'] = np.where((data['Stimuli'] == data['Responses']), 1, 0)

# rename tasks based on conditions - 0.5 -> low; 1 -> high
data['Task'] = np.where(data['Task'].eq("N") & data['Condition'].eq("Incongruent"), 'N_low', data['Task'])
data['Task'] = np.where(data['Task'].eq("N") & data['Condition'].eq("Congruent"), 'N_high', data['Task'])
data['Task'] = np.where(data['Task'].eq("C") & data['Condition'].eq("Incongruent"), 'C_low', data['Task'])
data['Task'] = np.where(data['Task'].eq("C") & data['Condition'].eq("Congruent"), 'C_high', data['Task'])

# get data in order by subject
data = data.sort_values(by=['Subject']).reset_index(drop=True)

In [18]:
getGroupData(data=data, nRatings=4, export='Xu_2019_Exp2')

N_low
------------
Done: Subject 1
Done: Subject 2
Done: Subject 3
Done: Subject 5
Done: Subject 8
Done: Subject 9
Done: Subject 10
Done: Subject 11
Done: Subject 12
Done: Subject 13
Done: Subject 14
Done: Subject 15
Done: Subject 17
Done: Subject 18
Done: Subject 19
Done: Subject 21
Done: Subject 24
Done: Subject 26
Done: Subject 27
------------
Completed: N_low
------------
C_high
------------
Done: Subject 1
Done: Subject 2
Done: Subject 3
Done: Subject 4
Done: Subject 5
Done: Subject 7
Done: Subject 9
Done: Subject 10
Done: Subject 11
Done: Subject 12
Done: Subject 13
Done: Subject 14
Done: Subject 15
Done: Subject 16
Done: Subject 17
Done: Subject 18
Done: Subject 19
Done: Subject 20
Done: Subject 22
Done: Subject 24
Done: Subject 26
Done: Subject 27
------------
Completed: C_high
------------
C_low
------------
Done: Subject 1
Done: Subject 2
Done: Subject 3
Done: Subject 5
Done: Subject 7
Done: Subject 9
Done: Subject 10
Done: Subject 11
Done: Subject 12
Done: Subject 13
Done: S

N_low_scores                                N_low_rates  \
Subject                                                               
1         (14, 9, 7, 14)   (0.6086956521739131, 0.3333333333333333)   
2        (16, 7, 10, 11)  (0.6956521739130435, 0.47619047619047616)   
3         (9, 12, 14, 7)  (0.42857142857142855, 0.6666666666666666)   
5         (8, 15, 11, 9)                (0.34782608695652173, 0.55)   
8        (15, 10, 6, 11)                 (0.6, 0.35294117647058826)   
9         (21, 4, 7, 13)                               (0.84, 0.35)   
10       (10, 14, 9, 10)  (0.4166666666666667, 0.47368421052631576)   
11        (9, 16, 12, 5)                 (0.36, 0.7058823529411765)   
12        (11, 8, 8, 13)  (0.5789473684210527, 0.38095238095238093)   
13       (12, 11, 6, 15)   (0.5217391304347826, 0.2857142857142857)   
14       (13, 10, 5, 17)  (0.5652173913043478, 0.22727272727272727)   
15       (11, 12, 13, 9)   (0.4782608695652174, 0.5909090909090909)   
17        (17, 5, 7, 16)  (0.7727272727272727, 0.30434782608695654)   
18       (9, 10, 14, 12)  (0.47368421052631576, 0.5384615384615384)   
19       (9, 10, 16, 10)  (0.47368421052631576, 0.6153846153846154)   
21        (6, 12, 16, 7)   (0.3333333333333333, 0.6956521739130435)   
24        (17, 5, 12, 9)   (0.7727272727272727, 0.5714285714285714)   
26        (13, 7, 5, 16)                (0.65, 0.23809523809523808)   
27       (9, 11, 13, 11)                 (0.45, 0.5416666666666666)   
4                    NaN                                        NaN   
7                    NaN                                        NaN   
16                   NaN                                        NaN   
20                   NaN                                        NaN   
22                   NaN                                        NaN   

         N_low_dprime  N_low_metad  N_low_criterion  N_low_mratio  \
Subject                                                             
1            0.674130     1.827049         0.077403      2.710231   
2            0.545776     1.651564        -0.226110      3.026084   
3           -0.581730    -0.381320        -0.125357      0.655492   
5           -0.493744     0.525298         0.132767     -1.063910   
8            0.598895     1.291699         0.062022      2.156805   
9            1.308182     1.996699        -0.304569      1.526316   
10          -0.139187     1.225689         0.138220     -8.806079   
11          -0.852591     1.314409        -0.091468     -1.541666   
12           0.477928     0.026852         0.051890      0.056185   
13           0.589764    -0.082323         0.255715     -0.139587   
14           0.866413     0.437126         0.291824      0.504523   
15          -0.271968     0.282280        -0.087683     -1.037915   
17           1.197879     0.732499        -0.117961      0.611496   
18          -0.155679     1.389233        -0.015273     -8.923721   
19          -0.344923    -0.095476        -0.113685      0.276803   
21          -0.895501    -0.416187        -0.040604      0.464753   
24           0.537355    -0.456171        -0.463935     -0.848918   
26           1.040596     1.893979         0.163561      1.820090   
27          -0.220082     0.413072         0.010514     -1.876900   
4                 NaN          NaN              NaN           NaN   
7                 NaN          NaN              NaN           NaN   
16                NaN          NaN              NaN           NaN   
20                NaN          NaN              NaN           NaN   
22                NaN          NaN              NaN           NaN   

         N_low_mdiff    C_high_scores  \
Subject                                 
1           1.152918   (18, 3, 3, 21)   
2           1.105788    (13, 0, 1, 7)   
3           0.200411   (17, 5, 2, 20)   
5           1.019042   (19, 5, 3, 17)   
8           0.692804              NaN   
9           0.688517   (17, 1, 7, 20)   
10          1.364876   (18, 4, 3, 17)   
1

In [37]:
getTotalTrialsAnalyzed(data=data, nRatings=4)

Completed: N_low
Completed: C_high
Completed: C_low
Completed: N_high
Number of subjects: 86
Number of recorded trials: 3955
Number of analyzed trials: 3693
Number of non-analyzed trials: 262
Subjects not analyzed (@ domain): ['Subject 4 @ N_low', 'Subject 7 @ N_low', 'Subject 20 @ N_low', 'Subject 22 @ N_low', 'Subject 21 @ C_high', 'Subject 4 @ C_low']
Percentage accounted for: 93.37547408343868%
Difference (%): 6.6245259165613195%


### `Ye_2018`

In [28]:
# read in data
data = pd.read_csv("../Confidence Database/data_Ye_2018.csv",
                   usecols=['Subj_idx', 'Stimulus', 'Response', 'Confidence', 'Task', 'Accuracy'])
data = data.rename(columns={"Subj_idx": "Subject", "Stimulus": "Stimuli", "Response": "Responses"})

# drop all NaNs
data = data.dropna().reset_index(drop=True)

# change dataset's 2s->1s and 1s->0s bc metadpy works with 0s and 1s
data['Stimuli'] = data['Stimuli'].map({1:0, 2:1})
data['Responses'] = data['Responses'].map({1:0, 2:1})

# get data in order by subject
data = data.sort_values(by=['Subject']).reset_index(drop=True)

In [20]:
getGroupData(data=data, nRatings=4, export='Ye_2018')

Perception
------------
Done: Subject 1
Done: Subject 2
Done: Subject 3
Done: Subject 4
Done: Subject 5
Done: Subject 6
Done: Subject 7
Done: Subject 8
Done: Subject 9
Done: Subject 10
Done: Subject 11
Done: Subject 12
Done: Subject 13
Done: Subject 14
Done: Subject 15
Done: Subject 16
Done: Subject 17
Done: Subject 18
------------
Completed: Perception
------------
Memory
------------
Done: Subject 1
Done: Subject 2
Done: Subject 3
Done: Subject 4
Done: Subject 5
Done: Subject 6
Done: Subject 7
Done: Subject 8
Done: Subject 9
Done: Subject 10
Done: Subject 11
Done: Subject 12
Done: Subject 13
Done: Subject 14
Done: Subject 15
Done: Subject 16
Done: Subject 17
Done: Subject 18
------------
Completed: Memory
------------
Exported: 'analysis_Ye_2018.csv'


perception_scores                           perception_rates  \
Subject                                                                   
1         (178, 31, 59, 187)  (0.8516746411483254, 0.23983739837398374)   
2         (183, 57, 66, 174)                            (0.7625, 0.275)   
3         (179, 46, 56, 193)   (0.7955555555555556, 0.2248995983935743)   
4         (179, 63, 48, 188)   (0.7396694214876033, 0.2033898305084746)   
5         (181, 59, 50, 184)  (0.7541666666666667, 0.21367521367521367)   
6         (178, 64, 59, 179)  (0.7355371900826446, 0.24789915966386555)   
7         (187, 50, 48, 194)  (0.7890295358649789, 0.19834710743801653)   
8         (193, 42, 54, 191)  (0.8212765957446808, 0.22040816326530613)   
9         (182, 47, 60, 190)                 (0.7947598253275109, 0.24)   
10        (152, 66, 90, 171)   (0.6972477064220184, 0.3448275862068966)   
11        (196, 73, 67, 138)  (0.7286245353159851, 0.32682926829268294)   
12        (189, 36, 37, 217)                (0.84, 0.14566929133858267)   
13       (180, 101, 59, 139)  (0.6405693950177936, 0.29797979797979796)   
14        (180, 61, 59, 178)   (0.7468879668049793, 0.2489451476793249)   
15        (205, 72, 38, 164)   (0.740072202166065, 0.18811881188118812)   
16        (179, 62, 64, 172)   (0.7427385892116183, 0.2711864406779661)   
17        (180, 80, 60, 157)   (0.6923076923076923, 0.2764976958525346)   
18        (186, 36, 59, 195)  (0.8378378378378378, 0.23228346456692914)   

         perception_dprime  perception_metad  perception_criterion  \
Subject                                                              
1                 1.739874          2.472283             -0.168409   
2                 1.305812          1.846765             -0.058304   
3                 1.573334          2.153811             -0.035050   
4                 1.464447          2.067944              0.093624   
5                 1.473871          2.055156              0.053037   
6                 1.304472          1.949050              0.025734   
7                 1.641955          2.643567              0.022241   
8                 1.682029          2.072194             -0.074713   
9                 1.521524          1.838358             -0.058373   
10                0.911639          1.198731             -0.058589   
11                1.052462          0.742474             -0.079986   
12                2.037425          2.050290              0.030366   
13                0.885942          0.844238              0.085119   
14                1.336032          1.645095              0.006542   
15                1.520075          2.092703              0.120641   
16                1.255023          1.835179             -0.021291   
17                1.090524          1.505301              0.045438   
18                1.707370          1.442211             -0.127131   

         perception_mratio  perception_mdiff        memory_scores  \
Subject                                                             
1                 1.420955          0.732409   (146, 84, 40, 186)   
2                 1.414265          0.540952   (149, 84, 93, 141)   
3                 1.368947          0.580477   (157, 75, 82, 155)   
4                 1.412099          0.603497   (155, 81, 60, 178)   
5                 1.394393          0.581285   (161, 78, 75, 163)   
6                 1.494129          0.644577  (180, 58, 102, 138)   
7                 1.610012          1.001612   (149, 88, 39, 197)   
8                 1.231961          0.390165   (174, 61, 65, 169)   
9                 1.208235          0.316835   (182, 51, 56, 175)   
10                1.314919          0.287092  (103, 112, 76, 128)   
11                0.705464         -0.309988  (127, 108, 75, 162)   
12                1.006314          0.012865   (173, 38, 34, 201)   
13                0.952927         -0.041704  (175, 60, 130, 105)   
14                1.231328          0.309062   (124, 99, 70, 154)   
15   

In [34]:
getTotalTrialsAnalyzed(data=data, nRatings=4)

Completed: perception
Completed: memory
Number of subjects: 36
Number of recorded trials: 16926
Number of analyzed trials: 16926
Number of non-analyzed trials: 0
Subjects not analyzed (@ domain): []
Percentage accounted for: 100.0%
Difference (%): 0.0%
